In [18]:
import pandas as pd
import sqlite3
import gdown

In [19]:
file_id = '1ocaBTiYrKkfnzsZoWtOpre5PV3X5R1ts'
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

gdown.download(download_url, 'Dataframe-camiones-limpio.txt', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1ocaBTiYrKkfnzsZoWtOpre5PV3X5R1ts
From (redirected): https://drive.google.com/uc?export=download&id=1ocaBTiYrKkfnzsZoWtOpre5PV3X5R1ts&confirm=t&uuid=fbecdc54-2fbc-451c-9206-f3d18a22d694
To: /content/Dataframe-camiones-limpio.txt
100%|██████████| 171M/171M [00:02<00:00, 63.3MB/s]


'Dataframe-camiones-limpio.txt'

In [20]:
df = pd.read_csv('Dataframe-camiones-limpio.txt', delimiter='|', encoding='latin-1')
df.sample(3)

,NATURALEZA,COD_PRODUCTO,PRODUCTO,CANTIDAD,UNID_MEDIDA,CODIGO_CARGUE,CARGUE,CODIGO_DESCARGUE,DESCARGUE,HORAS_VIAJE,...,VALOR_PACTADO,VALOR_PAGADO,CANTIDAD_REMESAS_VIAJE,EMPRESA_TRANSPORTE,PLACA,CONFIGURACION,CONDUCTOR,GPS,FECHA_SALIDA,FECHA_LLEGADA
425609,Carga Normal,1511,GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCT...,3450.0,Kilogramos,47053000,ARACATACA MAGDALENA,47001000,SANTA MARTA MAGDALENA,28.00,...,0.0,0.0,1,1036,100497,3S3,316168,0,2023-01-26 15:00:00,2023-01-27 19:00:00
119200,Carga Normal,9980,VARIOS PRODUCTOS VARIOS,9000.0,Kilogramos,68307000,GIRON SANTANDER,68077000,BARBOSA SANTANDER,23.00,...,900000.0,900000.0,1,391,269641,2,128453,0,2023-01-11 09:00:00,2023-01-12 08:00:00
271714,Carga Normal,7325,MANUFACTURAS DE FUNDICIÓN; DE HIERRO O DE ACER...,29000.0,Kilogramos,13001000,CARTAGENA BOLIVAR,23678000,SAN CARLOS CORDOBA,20.27,...,2800000.0,2800000.0,1,2148,346044,3S3,175825,0,2023-01-31 15:00:00,2023-02-01 11:16:00


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562090 entries, 0 to 562089
Data columns (total 24 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   NATURALEZA              562090 non-null  object 
 1   COD_PRODUCTO            562090 non-null  int64  
 2   PRODUCTO                562090 non-null  object 
 3   CANTIDAD                562090 non-null  float64
 4   UNID_MEDIDA             562090 non-null  object 
 5   CODIGO_CARGUE           562090 non-null  int64  
 6   CARGUE                  562090 non-null  object 
 7   CODIGO_DESCARGUE        562090 non-null  int64  
 8   DESCARGUE               562090 non-null  object 
 9   HORAS_VIAJE             562090 non-null  float64
 10  HORAS_ESPERA_CARGUE     562090 non-null  float64
 11  HORAS_CARGUE            562090 non-null  float64
 12  HORAS_ESPERA_DESCARGUE  562090 non-null  float64
 13  HORAS_DESCARGUE         562090 non-null  float64
 14  VALOR_PACTADO       

In [22]:
conn = sqlite3.connect('camiones.db')

df.to_sql('tabla_camiones', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

In [23]:
conn = sqlite3.connect('camiones.db')
query = "SELECT * FROM tabla_camiones LIMIT 5"
df_sql = pd.read_sql(query, conn)

print(df_sql)

     NATURALEZA  COD_PRODUCTO  \
0  Carga Normal          7323   
1  Carga Normal          7323   
2  Carga Normal          7323   
3  Carga Normal          7323   
4  Carga Normal          7323   

                                            PRODUCTO  CANTIDAD UNID_MEDIDA  \
0  MANUFACTURAS DE FUNDICIÓN DE HIERRO O DE ACERO...    7000.0  Kilogramos   
1  MANUFACTURAS DE FUNDICIÓN DE HIERRO O DE ACERO...    7000.0  Kilogramos   
2  MANUFACTURAS DE FUNDICIÓN DE HIERRO O DE ACERO...    7000.0  Kilogramos   
3  MANUFACTURAS DE FUNDICIÓN DE HIERRO O DE ACERO...    7000.0  Kilogramos   
4  MANUFACTURAS DE FUNDICIÓN DE HIERRO O DE ACERO...    7000.0  Kilogramos   

   CODIGO_CARGUE                        CARGUE  CODIGO_DESCARGUE  \
0       76109000  BUENAVENTURA VALLE DEL CAUCA          76520000   
1       76109000  BUENAVENTURA VALLE DEL CAUCA          76520000   
2       76109000  BUENAVENTURA VALLE DEL CAUCA          76520000   
3       76109000  BUENAVENTURA VALLE DEL CAUCA          7652

In [24]:
conn.execute('''
    CREATE TABLE IF NOT EXISTS Productos (
        COD_PRODUCTO INT PRIMARY KEY,
        DESCRIPCION TEXT
    )
''')

productos_df = df[['COD_PRODUCTO', 'PRODUCTO']].drop_duplicates()
productos_df.rename(columns={'PRODUCTO': 'DESCRIPCION'}, inplace=True)
productos_df.to_sql('Productos', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

In [25]:
conn = sqlite3.connect('camiones.db')

query = 'SELECT * FROM Productos LIMIT 15;'
result = conn.execute(query)

productos_df = pd.DataFrame(result.fetchall(), columns=[column[0] for column in result.description])

conn.close()
print(productos_df)

    COD_PRODUCTO                                        DESCRIPCION
0           7323  MANUFACTURAS DE FUNDICIÓN DE HIERRO O DE ACERO...
1           1511  GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCT...
2           1703  AZÚCARES Y ARTÍCULOS DE CONFITERÍA; MELAZA DE ...
3           1507  GRASAS Y ACEITES ANIMALES O VEGETALES; PRODUCT...
4           9880  OTROS MISCELANEOS CONTENIDOS EN PAQUETES ( PAQ...
5           9980                            VARIOS PRODUCTOS VARIOS
6           9403  MUEBLES; MOBILIARIO MEDICOQUIRÚRGICO; ARTÍCULO...
7           2704  COMBUSTIBLES MINERALES; ACEITES MINERALES Y PR...
8           3101  ABONOS; ABONOS DE ORIGEN ANIMAL O VEGETAL;  IN...
9           1005                                     CEREALES; MAIZ
10          8525  MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS P...
11          8547  MÁQUINA; APARATOS Y MATERIAL ELÉCTRICO Y SUS P...
12          9800  OTROS CONJUNTOS INDUSTRIALES EXPORTADOS DE CON...
13          9990                            VARI

In [26]:
conn = sqlite3.connect('camiones.db')

conn.execute('DROP TABLE IF EXISTS camiones;')

conn.execute('''
    CREATE TABLE camiones (
        EMPRESA_TRANSPORTE INT,
        PLACA TEXT PRIMARY KEY,
        CONFIGURACION TEXT,
        CONDUCTOR INT,
        GPS INT
    )
''')

df_camiones = df[['EMPRESA_TRANSPORTE', 'PLACA', 'CONFIGURACION', 'CONDUCTOR', 'GPS']]
df_camiones_unique = df_camiones.drop_duplicates(subset='PLACA')
df_camiones_unique.to_sql('camiones', conn, if_exists='append', index=False)


conn.commit()
conn.close()

In [30]:
conn = sqlite3.connect('camiones.db')

cursor = conn.cursor()

cursor.execute('''
select * from camiones
limit 10;''')

result = cursor.fetchall()

for row in result:
    print(row)

conn.commit()
conn.close()

(2148, '1400733', '2S2', 45, 0)
(2213, '91057', '3S3', 58, 0)
(2235, '1488863', '2', 106, 0)
(2154, '1190432', '2', 112, 0)
(1301, '220947', '3S3', 301, 0)
(297, '129929', '3S2', 304, 0)
(92975, '1017543', '2S3', 335, 0)
(391, '264017', '2', 1148, 0)
(1543, '217175', '3S3', 1191, 0)
(1012, '1535368', '3S3', 1215, 0)


In [28]:
conn = sqlite3.connect('camiones.db')
conn.execute('''
    update camiones
    set GPS = 1
    where PLACA = '133722'
''')

conn.commit()

cursor = conn.execute('select PLACA,GPS from camiones where PLACA = "133722";')
for row in cursor:
    print(row)

conn.close()

('133722', 1)


In [29]:
conn = sqlite3.connect('camiones.db')

conn.execute('''
    delete from camiones
    where PLACA = '133722'
''')

conn.commit()

cursor = conn.execute('select PLACA from camiones where PLACA = "133722";')
if cursor.fetchone() is None:
    print("Se ha borrado la fila.")
else:
    print("La fila aun existe.")

conn.close()

Se ha borrado la fila.


In [31]:
from google.colab import files
conn = sqlite3.connect('camiones.db')

conn.close()

files.download('camiones.db')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
conn = sqlite3.connect('camiones.db')

with open('camiones_dump.sql', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)

conn.close()

from google.colab import files
files.download('camiones_dump.sql')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>